In [ ]:
import pandas as pd
import glob
import os
import shutil
import re
from joblib import Parallel, delayed
import nibabel as nib
import numpy as np
import random
import seaborn as sns
from datetime import datetime

In [ ]:
import sys
sys.path.append('/media/raid/orco/data/ICH/code/neurorad_bids_utils')
from bids_utils.utils import utils, mask_utils

In [ ]:
dataset_names = ['Dataset011_5FoldCV', 'Dataset012_5FoldCV', 'Dataset013_5FoldCV', 'Dataset014_5FoldCV', 'Dataset015_5FoldCV']

In [ ]:
preds = glob.glob('/home/orco/data/VentrikelCNN/derivatives/nnUNet/nnUNet_raw/Dataset01*/test*/*.nii.gz')
len(preds)

In [ ]:
preds[0]

In [ ]:
df = pd.DataFrame(preds)
df.columns = ['prediction']
df['gt'] = df.prediction.apply(lambda x: x.replace('test_fold1', 'labelsTs'))
#df['fold'] = df.prediction.apply(lambda x: x.split('/')[-2].split('_')[-1])
df['ds'] = df.prediction.apply(lambda x: x.split('/')[-3])

In [ ]:
df.head()

In [ ]:
df.prediction[0]

In [ ]:
res = []
for i,j in zip(list(df.prediction),list(df['gt'])):
    res.append([i,j] + utils.mp_dice_n(i, j, [1,2,3,4,5]))

In [ ]:
df = pd.DataFrame(res)
df.head()

In [ ]:
len(df.columns)

In [ ]:
df.columns = ['prediction', 'gt', 'DSC_left', 'DSC_right', 'DSC_3rd', 'DSC_4th', 'DSC_Acq']


In [ ]:
#df['fold'] = df.prediction.apply(lambda x: x.split('/')[-2].split('_')[-1])
df['ds'] = df.prediction.apply(lambda x: x.split('/')[-3])

In [ ]:
df.head()

In [ ]:
df['sid'] = df.prediction.apply(lambda x: os.path.basename(x).split('_')[1].replace('.nii.gz',''))

In [ ]:
df.head()

In [ ]:
df = df[['sid','ds'] + [i for i in df.columns if i.startswith('DSC')] + ['gt', 'prediction']]

In [ ]:
date = '{}{}{}'.format(datetime.now().year, datetime.now().month, datetime.now().day)
date

In [ ]:
df.to_csv('./data/DSC_nnUNet_5classes_{}_{}.csv'.format('5FoldCV', date), index=False)